In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_format = 'retina'


In [ ]:
# Each row is an individual ZTF detection, each column is a field describing that detection
alerts = pd.concat((
    pd.read_csv("../BTS_data/train_cand_N100.csv", index_col=None),
    pd.read_csv("../BTS_data/val_cand_N100.csv", index_col=None),
    pd.read_csv("../BTS_data/test_cand_N100.csv", index_col=None)
))

# Each row is a ZTF source
# important columns here like redshift, type, etc.
sources = pd.read_csv("../BTS_data/explorer.csv", index_col=None)

In [ ]:
# we want some features that are not already present in the alerts
alerts[['redshift', 'type', 'A_V']] = -1

In [ ]:
alerts

In [ ]:
sources

In [ ]:
source = sources[sources["ZTFID"] == "ZTF22abgteub"]

In [ ]:
source['ZTFID']

In [ ]:
for s_idx in sources.index:
    obj_alerts = alerts.loc[alerts['objectId'] == sources.loc[s_idx, "ZTFID"]]
    
    # If this source has any alerts here,
    if len(obj_alerts) > 0:
        # Transfer necessary columns
        alerts.loc[alerts['objectId'] == sources.loc[s_idx, "ZTFID"], ["redshift", "type", "A_V"]] = sources.loc[s_idx, ["redshift", "type", "A_V"]]

In [ ]:
alerts.to_csv("../BTS_data/ZTF_alerts.csv", index=None)